# Summarization models in spanish
Following https://huggingface.co/josmunpen/mt5-small-spanish-summarization

A mirar https://huggingface.co/mrm8488/bert2bert_shared-spanish-finetuned-summarization

In [1]:
from datasets import load_dataset

In [2]:
dataset = load_dataset("quora")

Using custom data configuration default
Reusing dataset quora (/home/pablo/.cache/huggingface/datasets/quora/default/0.0.0/36ba4cd42107f051a158016f1bea6ae3f4685c5df843529108a54e42d86c1e04)


  0%|          | 0/1 [00:00<?, ?it/s]

## Duplicated questions
We check first the duplicated questions

In [3]:
for d in dataset["train"]:
    if d["is_duplicate"]:
        print(d)
        break

{'questions': {'id': [11, 12], 'text': ['Astrology: I am a Capricorn Sun Cap moon and cap rising...what does that say about me?', "I'm a triple Capricorn (Sun, Moon and ascendant in Capricorn) What does this say about me?"]}, 'is_duplicate': True}


In [4]:
ds = [d for d in dataset["train"] if d["is_duplicate"]]

In [5]:
len(ds)

149263

In [6]:
ds[7]

{'questions': {'id': [37, 38],
  'text': ['Why are so many Quora users posting questions that are readily answered on Google?',
   'Why do people ask Quora questions which can be answered easily by Google?']},
 'is_duplicate': True}

## Loading the model


In [1]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("josmunpen/mt5-small-spanish-summarization")
model = AutoModelForSeq2SeqLM.from_pretrained("josmunpen/mt5-small-spanish-summarization")

Downloading:   0%|          | 0.00/350 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/648 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.11M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/7.94M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

In [4]:
torch_device = "cpu"#'cuda' if torch.cuda.is_available() else 'cpu'
model = model.to(torch_device)

In [5]:
MAX_LEN = 5000
def get_response(input_text,num_return_sequences,num_beams):
  batch = tokenizer([input_text],truncation=True,padding='longest',max_length=MAX_LEN, return_tensors="pt").to(torch_device)
  translated = model.generate(**batch,max_length=100,num_beams=num_beams, num_return_sequences=num_return_sequences, temperature=1.5)
  tgt_text = tokenizer.batch_decode(translated, skip_special_tokens=True)
  return tgt_text

## Testing the model

In [17]:
num_beams = 10
num_return_sequences = 1
context = "El fin de la obligatoriedad de las mascarillas, que ha aprobado el martes el Consejo de Ministros y entra en vigor este miércoles, preocupa a la mitad de España. Un 48,5% se muestra “poco o nada seguro” cuando piensa en su día a día sin cubrebocas, frente a un 48,4% que está “muy o bastante seguro”, según una encuesta flash realizada a 500 personas por 40dB. para EL PAÍS y la Cadena SER entre el lunes y el martes. En lo que hay una clara mayoría es en el porcentaje de población que opina que se trata de una medida precipitada (un 54%, mientras un 28,2% que dice que es el momento adecuado y un 10,2% que expresa que llega tarde). También son mayoría los que seguirán usándola en los ámbitos donde dejará de ser obligatoria. Quien quiera abandonarla, podrá hacerlo en cualquier espacio interior, excepto en transportes públicos, discrecionales (como taxis), en servicios sanitarios de todo tipo, residencias (para visitantes y trabajadores) y farmacias, donde, por el momento, seguirá siendo obligatoria. A pesar de ello, el 70% cree bastante o muy probable seguir cubriéndose la cara en comercios, un porcentaje muy similar al de los que los seguirán haciendo en cines, teatros o exposiciones. Un 62,5% la seguirá llevando (bastante o muy probablemente) en centros deportivos o gimnasios, una cifra similar a la de bares y restaurantes y los centros de estudio o trabajo.  "

In [18]:
get_response(context,num_return_sequences,num_beams)

['el fin de la obligatoriedad de las mascarillas preocupa a la mitad de españa']